# Jacobian Derivation for BAL

## Set Up Session

In [1]:
# set up session:
from sympy import *
# init_printing(use_unicode=True)

## Define Variables

In [2]:
# projection in camera frame:
x, y, z = symbols('x y z')
# camera intrinsics:
fx, fy, cx, cy = symbols('fx fy cx cy')
# measurements:
u0, v0 = symbols('u0 v0')

## Camera Projection with Distortion

In [3]:
# projection on normalized plane:
x_prime = (x / z)
y_prime = (y / z)

# projection on pixel plane:
u = fx*x_prime + cx
v = fy*y_prime + cy

# error:
eu = u0 - u
ev = v0 - v

## Jacobian

In [4]:
# Jacobian of pixel plane projection with respect to projection on normalized plane:
J_p_prime = Matrix(
    [
        [diff(eu, x), diff(eu, y), diff(eu, z)],
        [diff(ev, x), diff(ev, y), diff(ev, z)]
    ]
)
# Jacobian of normalized plane projection with respect to se3:
J_lie = -Matrix(
    [
        [+0, -z, +y],
        [+z, +0, -x],
        [-y, +x, +0]
    ]
)

# dim 0~2 of CameraBAL:
J_rotation = simplify(J_p_prime * J_lie)
# dim 3~5 of CameraBAL:
J_translation = simplify(J_p_prime)

# final Jacobians:
J_xi = Matrix(
    [
        (J_rotation.row(0).tolist()[0] + J_translation.row(0).tolist()[0]),
        (J_rotation.row(1).tolist()[0] + J_translation.row(1).tolist()[0]),        
    ]
)
J_xj = simplify(J_p_prime)

## Convert to Ready-to-Use Format for G2O

In [5]:
def convert_to_g2o_code(J, vertex_name):
    """ convert matrix in sympy to g2o c++ code
    """
    result = []

    for i in range(2):
        desc = str(J.row(i).tolist()[0])
        # orders:
        for k in range(2, 7):
            # variables:
            for v in 'xyz':
                src = "{v}**{k}".format(v=v, k=k)
                dst = "{v}_{k}".format(v=v, k=k)

                desc = desc.replace(src, dst)
        
        result.append(
            "\n".join(
                [
                    "_jacobianOplusX{vertex_name}({i},{j}) = {v};".format(
                        vertex_name=vertex_name,
                        i=i, j=j, 
                        v=v
                    ) 
                    for j,v in enumerate(desc[1:-1].split(','))
                ]
            )
        )

    return "\n\n".join(result)

In [6]:
# vertex_camera:
print(convert_to_g2o_code(J_xi, 'i'))

_jacobianOplusXi(0,0) = fx*x*y/z_2;
_jacobianOplusXi(0,1) =  -fx*x_2/z_2 - fx;
_jacobianOplusXi(0,2) =  fx*y/z;
_jacobianOplusXi(0,3) =  -fx/z;
_jacobianOplusXi(0,4) =  0;
_jacobianOplusXi(0,5) =  fx*x/z_2;

_jacobianOplusXi(1,0) = fy*y_2/z_2 + fy;
_jacobianOplusXi(1,1) =  -fy*x*y/z_2;
_jacobianOplusXi(1,2) =  -fy*x/z;
_jacobianOplusXi(1,3) =  0;
_jacobianOplusXi(1,4) =  -fy/z;
_jacobianOplusXi(1,5) =  fy*y/z_2;


In [7]:
# vertex_point, partial:
print(convert_to_g2o_code(J_xj, 'j'))

_jacobianOplusXj(0,0) = -fx/z;
_jacobianOplusXj(0,1) =  0;
_jacobianOplusXj(0,2) =  fx*x/z_2;

_jacobianOplusXj(1,0) = 0;
_jacobianOplusXj(1,1) =  -fy/z;
_jacobianOplusXj(1,2) =  fy*y/z_2;
